In [205]:
%pip install requests beautifulsoup4 pandas lxml

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [206]:
%pip install selenium

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [207]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np

In [208]:
URL = "https://www.ssa.ingenieria.unam.mx/horarios.html"
page = requests.get(URL)

In [209]:
soup = BeautifulSoup(page.content, 'lxml')
form = soup.find('div', class_='input-group')

In [210]:
driver = webdriver.Chrome()
driver.get(URL)

In [211]:
radio_button = driver.find_element(By.CSS_SELECTOR, 'input[name="optHorarioAsignatura"]')
radio_button.click()

In [212]:
input_field = driver.find_element(By.ID, 'clave')
input_field.send_keys('6867')
input_field.submit()

In [213]:
# Diccionario para mapear los días a números
days_map = {'Lun': 0, 'Mar': 1, 'Mie': 2, 'Jue': 3, 'Vie': 4, 'Sab': 5, 'Dom': 6}

def convert_days(days):
    return ', '.join([str(days_map[day]) for day in days.split(', ')])

# List of tables
tables = driver.find_elements(By.CLASS_NAME, 'table-horarios-custom')

# New list to append the rows
rows_list = []

for table in tables:
    rows = table.find_elements(By.TAG_NAME, 'tr')
    
    for row in rows:
        cells = [cell.text for cell in row.find_elements(By.TAG_NAME, 'td')]
        rows_list.append(cells)

# Delete the empty elements from rows_list
rows_list = [row for row in rows_list if row]

# # If a row has 'L' in the first position, it means that it is from the previous group, so we need to add the group data to the previous row
# For example using this group: ['1858', '1', 'ING. LUIS ALEJANDRO DE LA CRUZ VELAZQUEZ\n(PRESENCIAL)', 'T', '07:00 a 08:30', 'Lun, Mie', 'B206', '35', '32']
# The first row will be: ['1858', '1', 'ING. LUIS ALEJANDRO DE LA CRUZ VELAZQUEZ\n(PRESENCIAL)', 'T', '07:00 a 08:30', '0, 2', 'B206', '35', '32']
# After that, the next row is ['L', '07:00 a 09:00', 'Mar', 'T101']
# So we need to add the group data to the previous row, so it will be: ['1858', '1', 'ING. LUIS ALEJANDRO DE LA CRUZ VELAZQUEZ\n(PRESENCIAL)', 'L', '07:00 a 09:00', 'Mar', 'T101']
# Busca el índice de la lista que comienza con 'L'
index = next((i for i, x in enumerate(rows_list) if x[0] == 'L'), None)

while index is not None:
    # Obtiene la lista anterior
    prev_list = rows_list[index - 1]
    
    # Crea una nueva lista que contenga los primeros tres elementos de la lista anterior y los elementos de la lista original
    new_list = prev_list[:3] + rows_list[index]
    
    # Reemplaza la lista original con la nueva lista
    rows_list[index] = new_list
    
    # Busca el próximo índice de la lista que comienza con 'L'
    index = next((i for i, x in enumerate(rows_list) if x[0] == 'L' and i > index), None)

print(rows_list)

df = pd.DataFrame(rows_list)

# Rename the columns
df.columns = ['Clave', 'Grupo', 'Profesor', 'Tipo', 'Horario', 'Días', 'Salón', 'Cupo', 'Vacantes']

# Drop the columns "Vacantes" and "Cupo

df = df.drop(columns=['Vacantes', 'Cupo'])

# Delete the empty rows
df = df.replace('', np.nan)
df = df.dropna()

# Split the professor and the modality into two columns
df[['Profesor', 'Modalidad']] = df['Profesor'].str.split('\n', expand=True)

# Reset the index from 0 to n
df.reset_index(drop=True, inplace=True)

# Split the "Horario" column into two columns: "INI" y "FIN".
df[['INI', 'FIN']] = df['Horario'].str.split(' a ', expand=True)

# Elimina la columna original 'Horario'
df = df.drop(columns=['Horario'])

# Replace the "Días" column with the days of the week starting from Monday = 0 to Sunday = 6. For example "Mar, Jue" = "1, 3"
df['Días'] = df['Días'].apply(convert_days)

df

[['6867', '1', 'ING. ROSA ELENA LOBERA SANCHEZ\n(PRESENCIAL)', 'L+', '07:00 a 09:00', 'Mar', 'Q209', '22', '18'], ['6867', '2', 'M.I. LUIS SERGIO DURAN ARENAS\n(PRESENCIAL)', 'L+', '15:00 a 17:00', 'Mar', 'Q209', '22', '1'], ['6867', '3', 'ING. ROSA ELENA LOBERA SANCHEZ\n(PRESENCIAL)', 'L+', '07:00 a 09:00', 'Mie', 'Q209', '22', '12'], ['6867', '5', 'M.I. LUIS SERGIO DURAN ARENAS\n(PRESENCIAL)', 'L+', '17:00 a 19:00', 'Jue', 'Q209', '22', '6'], ['6867', '4', 'M.I. PEDRO IGNACIO RINCON GOMEZ\n(PRESENCIAL)', 'L+', '17:00 a 19:00', 'Mie', 'Q209', '22', '0'], ['6867', '6', 'ING. JULIO CESAR CRUZ ESTRADA\n(PRESENCIAL)', 'L+', '07:00 a 09:00', 'Vie', 'Q209', '22', '0'], ['6867', '7', 'M.A. AYESHA SAGRARIO ROMAN GARCIA\n(PRESENCIAL)', 'L+', '20:00 a 22:00', 'Vie', 'Q209', '22', '0'], ['6867', '8', 'MTRO. JORGE FERAT TOSCANO\n(PRESENCIAL)', 'L+', '07:00 a 09:00', 'Sab', 'Q209', '22', '0']]


,Clave,Grupo,Profesor,Tipo,Días,Salón,Modalidad,INI,FIN
0,6867,1,ING. ROSA ELENA LOBERA SANCHEZ,L+,1,Q209,(PRESENCIAL),07:00,09:00
1,6867,2,M.I. LUIS SERGIO DURAN ARENAS,L+,1,Q209,(PRESENCIAL),15:00,17:00
2,6867,3,ING. ROSA ELENA LOBERA SANCHEZ,L+,2,Q209,(PRESENCIAL),07:00,09:00
3,6867,5,M.I. LUIS SERGIO DURAN ARENAS,L+,3,Q209,(PRESENCIAL),17:00,19:00
4,6867,4,M.I. PEDRO IGNACIO RINCON GOMEZ,L+,2,Q209,(PRESENCIAL),17:00,19:00
5,6867,6,ING. JULIO CESAR CRUZ ESTRADA,L+,4,Q209,(PRESENCIAL),07:00,09:00
6,6867,7,M.A. AYESHA SAGRARIO ROMAN GARCIA,L+,4,Q209,(PRESENCIAL),20:00,22:00
7,6867,8,MTRO. JORGE FERAT TOSCANO,L+,5,Q209,(PRESENCIAL),07:00,09:00


In [214]:
df.to_csv('output.csv', index=False)

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import numpy as np

# Diccionario para mapear los días a números
days_map = {'Lun': 0, 'Mar': 1, 'Mie': 2, 'Jue': 3, 'Vie': 4, 'Sab': 5, 'Dom': 6}

# Función para convertir los días a números
def convert_days(days):
    return ', '.join([str(days_map[day]) for day in days.split(', ')])

# Import the courses_and_key csv file
df = pd.read_csv('courses_and_keys.csv')
df

# Create a dictionary with the courses and keys
courses_and_key = dict(zip(df['course_name'], df['key']))
courses_and_key

# Now, for every subject in the dictionary, we will retrieve their schedules

# Prepare the URL and Selenium driver
URL = "https://www.ssa.ingenieria.unam.mx/horarios.html"
driver = webdriver.Chrome()
driver.get(URL)

# Select the radio button "Horario por asignatura"
radio_button = driver.find_element(By.CSS_SELECTOR, 'input[name="optHorarioAsignatura"]')
radio_button.click()

for key in courses_and_key:    

    # Select the input field, clear it, and send the key
    input_field = driver.find_element(By.ID, 'clave')
    input_field.clear()
    input_field.send_keys(courses_and_key[key])
    input_field.submit()

    # List of tables
    tables = driver.find_elements(By.CLASS_NAME, 'table-horarios-custom')

    # New list to append the rows
    rows_list = []

    for table in tables:
        rows = table.find_elements(By.TAG_NAME, 'tr')
        
        for row in rows:
            cells = [cell.text for cell in row.find_elements(By.TAG_NAME, 'td')]
            rows_list.append(cells)

    # Delete the empty elements from rows_list
    rows_list = [row for row in rows_list if row]

    index = next((i for i, x in enumerate(rows_list) if x[0] == 'L'), None)

    while index is not None:
        # Obtiene la lista anterior
        prev_list = rows_list[index - 1]
        
        # Crea una nueva lista que contenga los primeros tres elementos de la lista anterior y los elementos de la lista original
        new_list = prev_list[:3] + rows_list[index]
        
        # Reemplaza la lista original con la nueva lista
        rows_list[index] = new_list
        
        # Busca el próximo índice de la lista que comienza con 'L'
        index = next((i for i, x in enumerate(rows_list) if x[0] == 'L' and i > index), None)

    df = pd.DataFrame(rows_list)

    # Rename the columns
    df.columns = ['Clave', 'Grupo', 'Profesor', 'Tipo', 'Horario', 'Días', 'Salón', 'Cupo', 'Vacantes']

    # Drop the columns "Vacantes" and "Cupo

    df = df.drop(columns=['Vacantes', 'Cupo'])

    # Delete the empty rows
    df = df.replace('', np.nan)
    df = df.dropna()

    # Split the professor and the modality into two columns
    df[['Profesor', 'Modalidad']] = df['Profesor'].str.split('\n', expand=True)

    # Reset the index from 0 to n
    df.reset_index(drop=True, inplace=True)

    # Split the "Horario" column into two columns: "INI" y "FIN".
    df[['INI', 'FIN']] = df['Horario'].str.split(' a ', expand=True)

    # Elimina la columna original 'Horario'
    df = df.drop(columns=['Horario'])

    # Replace the "Días" column with the days of the week starting from Monday = 0 to Sunday = 6. For example "Mar, Jue" = "1, 3"
    df['Días'] = df['Días'].apply(convert_days)

    # Export the dataframe to a csv file with the name of the course in the folder "output"
    df.to_csv(f'results/{key}.csv', index=False)

    # Cerrar el driver
    driver.quit()

AttributeError: 'NoneType' object has no attribute 'clear'